In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats
from sklearn import preprocessing
%matplotlib inline
from sklearn.feature_selection import SequentialFeatureSelector as sfs
from scipy import stats
from scipy.stats import linregress
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from metric_utils import calculate_metri

In [2]:
# Import dataset
train = pd.read_csv('../data/trainProcessed.csv')
validate = pd.read_csv('../data/validateProcessed.csv')
test = pd.read_csv('../data/testProcessed.csv')

In [3]:
#Our group decided to use Random forest to predict the pathology, drop unecessary columns.

def data_pre(df):
    y=df['PATHOLOGY']
    X=df.drop(['DIFFERENTIAL_DIAGNOSIS','PATHOLOGY','INITIAL_EVIDENCE'], axis=1)
    return X,y
X,y=data_pre(train)

In [4]:
X,y=data_pre(train)

In order to get the best parameter of the model, our group decided to use grid search.

In [10]:
# To find the best parameter for the model, we decided to run the grid search.

X,y=data_pre(train)
param_grid = {
    'n_estimators':[100,300],
    'max_depth': [10, 20],
    'min_samples_split': [2,4],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'auto'],
}


myclf = RandomForestClassifier(random_state=0)


grid_search = GridSearchCV(estimator=myclf, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

grid_search.fit(X, y)

best_params_ = grid_search.best_params_


Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=434.8min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=300; total time=494.3min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=100; total time=23.5min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=100; total time=23.2min
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=22.1min
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=22.7min
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=64.1min
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=4, n_estimators=

[CV] END max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=4, n_estimators=100; total time=147.8min
[CV] END max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=4, n_estimators=300; total time=115.4min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=420.5min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=300; total time=505.6min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=66.0min
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=23.2min
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=23.5min
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=22.4min
[CV] END max_depth=10, max_features=

[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=104.2min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=478.9min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=264.3min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=125.4min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=300; total time=67.7min
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=23.8min
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=4, n_estimators=300; total time=65.1min
[CV] END max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=65.0min
[CV] END max_depth=10, max_features=

After finding the best parameter, we use it in the model and see the performance on the valifation set.

In [11]:
clf = RandomForestClassifier(**best_params_,
#                               n_estimators = 300,
                             class_weight='balanced',
#                              max_features= 'auto',
                             criterion='entropy',
                             bootstrap=False,
                             warm_start=True,
                             random_state=0)
clf.fit(X, y)

/Users/zihanzhao/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:765: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(


RandomForestClassifier(bootstrap=False, class_weight='balanced',
                       criterion='entropy', max_depth=20, max_features='sqrt',
                       n_estimators=300, random_state=0, warm_start=True)

**Evaluate on validation dataset**

In [12]:
# Prepare the validation set
X_validate,y_validate=data_pre(validate)
y_pred_validate=clf.predict(X_validate)

In [13]:
# Use Accuracy, Precision, Recall and F1 score to evaluate the performance
accuracy_v = accuracy_score(y_validate, y_pred_validate)
precision_v = precision_score(y_validate, y_pred_validate,average='macro')
recall_v = recall_score(y_validate, y_pred_validate,average='macro')
f1 = f1_score(y_validate, y_pred_validate, average='weighted')

print(f"Accuracy: {accuracy_v}")
print(f"Precision: {precision_v}")
print(f"Recall: {recall_v}")
print(f"F1 Score: {f1}")

Accuracy: 0.9967969298875148
Precision: 0.9970599332940475
Recall: 0.9958753203048871
F1 Score: 0.9967513319074114


According to the model's outcome validation set, the Accuracy and F1 score is high enough, so it is good to use in the test set. The reason of its high accuracy and F1 score is because of the enormous size of our dataset.

**Result on Test dataset**

In [14]:
# Prepare the test set
y_test=test['PATHOLOGY']
X_test=test.drop(['DIFFERENTIAL_DIAGNOSIS','PATHOLOGY','INITIAL_EVIDENCE'], axis=1)
y_pred_test=clf.predict(X_test)

In [15]:
# apply the function to get the evaluation metric
test_metric = calculate_metric(y_test, y_pred_test, index=["data"])
test_metric = test_metric.loc['data']

print(f"Accuracy: {test_metric['accuracy']}")
print(f"Precision: {test_metric['precision']}")
print(f"Recall: {test_metric['recall']}")
print(f"F1 Score: {test_metric['f1 score']}")
print(f"Balanced Accuracy: {test_metric['balanced accuracy']}")

Accuracy: 0.9972252804475258
Precision: 0.9973300992540448
Recall: 0.9963547832020998
F1 Score: 0.9966679090285613
Balanced Accuracy: 0.9963547832020998


Our final result on the test set is shown above. We reached the maximum accuracy of 0.9972 and 0.9967 F1 score, which is high enough to predict the pathology.

**Accuracy for each pathology**

In [16]:
classes = np.unique(y_test)
accuracies = {}
for cls in classes:
    cls_true = (y_test == cls).astype(int)
    cls_pred = (y_pred_test == cls).astype(int)

    accuracies[cls] = accuracy_score(cls_true, cls_pred)

for cls, acc in accuracies.items():
    print(f"Accuracy for class {cls}: {acc}")

Accuracy for class Anaphylaxie: 1.0
Accuracy for class Angine instable: 0.9996131758264647
Accuracy for class Angine stable: 0.9996131758264647
Accuracy for class Anémie: 1.0
Accuracy for class Asthme exacerbé ou bronchospasme: 1.0
Accuracy for class Attaque de panique: 1.0
Accuracy for class Bronchiectasies: 1.0
Accuracy for class Bronchiolite: 1.0
Accuracy for class Bronchite: 1.0
Accuracy for class Chagas: 1.0
Accuracy for class Coqueluche: 1.0
Accuracy for class Céphalée en grappe: 1.0
Accuracy for class Ebola: 1.0
Accuracy for class Embolie pulmonaire: 1.0
Accuracy for class Exacerbation aigue de MPOC et/ou surinfection associée: 1.0
Accuracy for class Fibrillation auriculaire/Flutter auriculaire: 1.0
Accuracy for class Fracture de côte spontanée: 1.0
Accuracy for class Hernie inguinale: 1.0
Accuracy for class IVRS ou virémie: 1.0
Accuracy for class Laryngite aigue: 0.9997545154283334
Accuracy for class Laryngo-trachéo-bronchite (Croup): 0.9999925610735858
Accuracy for class Laryn


However, there is one limitation: the grid search process takes too long, our group is still trying to find a less time consuming way. Also, our dataset only contains pathology that are related with breath, throat and lungth. The model performance on dataset that contain wider pathology range is still unclear.